In [1]:
import os
import splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Conv2D, MaxPool2D, Flatten, Dropout

In [ ]:
# 미디어 파이프 설정 변수 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,min_detection_confidence=0.3, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils

In [ ]:
# 포즈 감지 함수
def detectPose(image,pose):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    results = pose.process(imageRGB)
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
    return output_image

In [ ]:
# 점찍힌 사진 만들기
actions = ['bend','headup','right','waistup']
for idx, action in enumerate(actions):
    for i in range(len(os.listdir(f'./CNN_dataset/{action}'))):
        image = cv2.imread(f"./CNN_dataset/{action}/{os.listdir(f'./CNN_dataset/{action}')[i]}")
        output_image = detectPose(image)
        cv2.imwrite(f'./CNN_dataset2/{action}/{action,idx}.jpg',output_image)


In [2]:
# 폴더안의 데이터를 train, test, val로 나누기 -> dataset 폴더안에 나뉘어짐
splitfolders.ratio("./CNN_dataset/",output="./4pose_dataset/",seed=1337,ratio=(.8,.1,.1))

In [3]:
# 각각 폴더 경로 선언
train_dir = './4pose_dataset/train/'
test_dir = './4pose_dataset/test/'
val_dir = './4pose_dataset/val/'

In [4]:
# 폴더 전처리 한번에 -> ImageDataGenerator
generator = ImageDataGenerator(rescale = 1./255)
train_generator = generator.flow_from_directory(
    directory = train_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
test_generator = generator.flow_from_directory(
    directory = test_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
val_generator = generator.flow_from_directory(
    directory = val_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
# 다중분류 -> categorical 
print(train_generator.class_indices)

Found 2292 images belonging to 4 classes.
Found 290 images belonging to 4 classes.
Found 284 images belonging to 4 classes.
{'bend': 0, 'headup': 1, 'right': 2, 'waistup': 3}


In [5]:
# CNN 모델 선언
model = Sequential()
# 입력층 , Conv층 
model.add(Conv2D(
    filters = 32 , 
    kernel_size = (3,3),
    input_shape = (150,150,3),
    padding='same',
    activation = 'relu'
))

# Pool층
model.add(MaxPool2D(
    pool_size= (2,2)
))
# 특징 과대적합 방지 -> dropout 
model.add(Dropout(0.3))

model.add(Conv2D(
    filters = 16 ,
    kernel_size = (3,3),
    padding='same',
    activation = 'relu'
))
model.add(MaxPool2D(
    pool_size=(2,2) 
))
model.add(Dropout(0.3))
# 특징 추출 끝
# 1차원으로 펼치기 
model.add(Flatten())
# 시작 
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=16, activation='relu'))
# 출력 - units -> 특성수 = 4 , 다중분류 -> softmax
model.add(Dense(units=4,activation='softmax'))

In [6]:
# 모델 학습과정 설정하기
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [7]:
#학습
model.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

C:\Users\smhrd\AppData\Local\Temp\ipykernel_10640\3927506946.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
23/23 [==============================] - 56s 2s/step - loss: 1.2868 - accuracy: 0.6169 - val_loss: 0.9883 - val_accuracy: 0.6725
Epoch 2/20
23/23 [==============================] - 41s 2s/step - loss: 0.8442 - accuracy: 0.6675 - val_loss: 0.9090 - val_accuracy: 0.6725
Epoch 3/20
23/23 [==============================] - 41s 2s/step - loss: 0.7923 - accuracy: 0.6649 - val_loss: 0.7815 - val_accuracy: 0.6725
Epoch 4/20
23/23 [==============================] - 41s 2s/step - loss: 0.7436 - accuracy: 0.6658 - val_loss: 0.7690 - val_accuracy: 0.7148
Epoch 5/20
23/23 [==============================] - 41s 2s/step - loss: 0.7075 - accuracy: 0.6702 - val_loss: 0.7089 - val_accuracy: 0.7113
Epoch 6/20
23/23 [==============================] - 41s 2s/step - loss: 0.6490 - accuracy: 0.6990 - val_loss: 0.6142 - val_accuracy: 0.7394
Epoch 7/20
23/23 [==============================] - 41s 2s/step - loss: 0.5857 - accuracy: 0.7334 - val_loss: 0.5772 - val_accuracy: 0.7606
Epoch 8/20
23/23 [==

In [8]:
# 저장
from keras.models import load_model
model.save('01.Cnn_model.h5')